# 📊 Proyecto: Analysis.py — Notebook para Presentación

Este notebook reproduce y documenta paso a paso la lógica de tu aplicación `src/analysis.py`.
Usa **rutas relativas** (recomendado) para que puedas mover el proyecto o presentarlo sin errores.

**Estructura de carpetas esperada (relativa a este notebook):**

- `DataBase/` → archivos: `clientes.xlsx`, `detalle_ventas.xlsx`, `productos.xlsx`, `ventas.xlsx`
- `outputs/` → se crearán subcarpetas `figures/` y `reports/`

---
### ¿Qué encontrarás en este notebook?
- Carga de datos y validación de columnas
- Limpieza mínima y cálculo de `venta_total`
- Análisis descriptivo (estadísticas, outliers, correlaciones)
- Análisis por categoría (si existe `productos.xlsx`)
- Gráficas listas para presentar (guardadas en `outputs/figures/`)
- Conclusiones resumidas y recomendaciones para presentar


In [ ]:
# 🔧 1) Configuración de entorno y rutas (USAR RELATIVAS)
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
PROJECT_ROOT = Path.cwd()
DB_DIR = PROJECT_ROOT / "DataBase"
OUT_DIR = PROJECT_ROOT / "outputs"
FIG_DIR = OUT_DIR / "figures"
REPORTS_DIR = OUT_DIR / "reports"

# Crear carpetas de salida si no existen
FIG_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

DB_DIR, OUT_DIR, FIG_DIR, REPORTS_DIR


In [ ]:
# 📥 2) Carga de datos (verifica nombres de archivos en DataBase/)
def safe_read_excel(path):
    if path.exists():
        try:
            df = pd.read_excel(path, engine='openpyxl')
            print(f"✅ Cargado: {path.name} ({len(df)} filas)")
            return df
        except Exception as e:
            print(f"❌ Error leyendo {path.name}: {e}")
            return None
    else:
        print(f"⚠️ No encontrado: {path}")
        return None

clientes = safe_read_excel(DB_DIR / "clientes.xlsx")
productos = safe_read_excel(DB_DIR / "productos.xlsx")
ventas = safe_read_excel(DB_DIR / "ventas.xlsx")
detalle = safe_read_excel(DB_DIR / "detalle_ventas.xlsx")

# Mostrar primeras filas para validar
for name, df in [("clientes", clientes), ("productos", productos), ("ventas", ventas), ("detalle", detalle)]:
    if df is not None:
        display(df.head(2))


In [ ]:
# 🔁 3) Preprocesamiento mínimo y normalización de nombres de columna
import unicodedata

def normalize_col_name(s: str) -> str:
    s = str(s).strip().lower()
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    s = s.replace(' ', '_')
    return s

def normalize_df_cols(df):
    df = df.copy()
    df.columns = [normalize_col_name(c) for c in df.columns]
    return df

# Normalizar columnas
if clientes is not None: clientes = normalize_df_cols(clientes)
if productos is not None: productos = normalize_df_cols(productos)
if ventas is not None: ventas = normalize_df_cols(ventas)
if detalle is not None: detalle = normalize_df_cols(detalle)

print('Columnas después de normalizar:')
for name, df in [("clientes", clientes), ("productos", productos), ("ventas", ventas), ("detalle", detalle)]:
    if df is not None:
        print(f"- {name}: {list(df.columns)[:20]}")


In [ ]:
# 🔢 4) Calcular `venta_total` si no existe y asegurar tipos numéricos
# Buscamos columnas plausibles para cantidad y precio en detalle
if detalle is None:
    raise FileNotFoundError("detalle_ventas.xlsx no cargado. No se puede continuar.")

# Identificar columnas
qty_candidates = ['cantidad','cantidad_vendida','qty','quantity']
price_candidates = ['precio_unitario','precio','price','unit_price']
def find_col(df, candidates):
    cols = {c: c for c in df.columns}
    for cand in candidates:
        for col in df.columns:
            if cand == col:
                return col
    for col in df.columns:
        for cand in candidates:
            if cand in col:
                return col
    return None

qty_col = find_col(detalle, qty_candidates)
price_col = find_col(detalle, price_candidates)

print("qty_col:", qty_col, "price_col:", price_col)

# Calcular venta_total si hace falta
if 'venta_total' not in detalle.columns and qty_col and price_col:
    detalle['venta_total'] = pd.to_numeric(detalle[qty_col], errors='coerce') * pd.to_numeric(detalle[price_col], errors='coerce')
    print("✅ Calculada columna 'venta_total' en detalle.")
elif 'venta_total' in detalle.columns:
    detalle['venta_total'] = pd.to_numeric(detalle['venta_total'], errors='coerce')
else:
    print("⚠️ No se pudo determinar 'venta_total'. Revisa columnas de cantidad y precio.")

In [ ]:
# 🔗 5) Merge con productos para obtener nombre y categoria (si existe)
df_merged = detalle.copy()
if productos is not None and 'id_producto' in productos.columns and 'id_producto' in df_merged.columns:
    df_merged = df_merged.merge(productos[['id_producto','nombre_producto','categoria']], on='id_producto', how='left', suffixes=('','_prod'))
    # preferir nombre_producto_prod si existe
    if 'nombre_producto_prod' in df_merged.columns:
        df_merged['nombre_producto'] = df_merged['nombre_producto_prod']
        df_merged.drop(columns=['nombre_producto_prod'], inplace=True)
    print("✅ Merge ejecutado. Columnas ahora:", list(df_merged.columns)[:30])
else:
    print("ℹ️ products.xlsx no disponible o columna id_producto falta; se omite merge.")

In [ ]:
# 📊 6) Estadísticas descriptivas clave
def describe_series(s):
    return {
        'count': int(s.count()),
        'mean': float(s.mean()) if s.count()>0 else None,
        'median': float(s.median()) if s.count()>0 else None,
        'std': float(s.std()) if s.count()>0 else None,
        'min': float(s.min()) if s.count()>0 else None,
        'max': float(s.max()) if s.count()>0 else None,
        'skew': float(s.skew()) if s.count()>0 else None
    }

target_cols = {}
# Detectar columnas
for col in ['precio_unitario','precio','price','unit_price']:
    if col in df_merged.columns:
        target_cols['precio_unitario'] = col
        break
for col in ['cantidad','cantidad_vendida','qty','quantity']:
    if col in df_merged.columns:
        target_cols['cantidad'] = col
        break
for col in ['venta_total','total_venta','total','importe','total_amount']:
    if col in df_merged.columns:
        target_cols['total_venta'] = col
        break

print('Target cols detectadas:', target_cols)
for label, col in target_cols.items():
    print('\n---', label, '(', col, ') ---')
    print(describe_series(pd.to_numeric(df_merged[col], errors='coerce')))

In [ ]:
# 📈 7) Distribuciones, sesgo y correlaciones
import numpy as np

for label, col in target_cols.items():
    s = pd.to_numeric(df_merged[col], errors='coerce')
    print(f"{label}: count={s.count()}, mean={s.mean():.2f}, skew={s.skew():.2f}")

# Correlation matrix
num_cols = [target_cols[k] for k in target_cols]
corr = df_merged[num_cols].dropna().corr()
display(corr)

# Guardar heatmap
plt.figure(figsize=(6,5))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r')
plt.title('Matriz de Correlaciones')
plt.tight_layout()
plt.savefig(FIG_DIR/'heatmap_correlaciones_notebook.png', dpi=200)
plt.show()
print('Guardado en:', FIG_DIR/'heatmap_correlaciones_notebook.png')

In [ ]:
# 📚 8) Análisis por categoría (si está disponible)
if 'categoria' in df_merged.columns:
    cats = df_merged['categoria'].fillna('SIN_CAT').unique()
    summary = []
    for c in cats:
        dfc = df_merged[df_merged['categoria']==c]
        ingresos = dfc[target_cols['total_venta']].sum()
        trans = len(dfc)
        prod_unicos = dfc['id_producto'].nunique() if 'id_producto' in dfc.columns else None
        summary.append((c, trans, prod_unicos, ingresos))
    summary_df = pd.DataFrame(summary, columns=['categoria','transacciones','productos_unicos','ingresos_totales']).sort_values('ingresos_totales', ascending=False)
    display(summary_df)
    # Guardar gráfico sencillo
    plt.figure(figsize=(10,6))
    plt.bar(summary_df['categoria'], summary_df['ingresos_totales'])
    plt.xticks(rotation=45, ha='right')
    plt.title('Ingresos por Categoria')
    plt.tight_layout()
    plt.savefig(FIG_DIR/'ingresos_por_categoria_notebook.png', dpi=200)
    plt.show()
    print('Guardado:', FIG_DIR/'ingresos_por_categoria_notebook.png')
else:
    print('No hay columna categoria; omitiendo análisis por categoría.')

In [ ]:
# 🏆 9) Top 10 productos y distribuciones por producto
# Top 10
if 'nombre_producto' in df_merged.columns:
    top10 = df_merged.groupby('nombre_producto')[target_cols['total_venta']].sum().nlargest(10)
else:
    top10 = df_merged.groupby('id_producto')[target_cols['total_venta']].sum().nlargest(10)

plt.figure(figsize=(12,6))
top10.plot(kind='bar', edgecolor='black')
plt.title('Top 10 Productos por Venta Total')
plt.ylabel('Venta Total')
plt.tight_layout()
plt.savefig(FIG_DIR/'top10_productos_notebook.png', dpi=200)
plt.show()
print('Guardado:', FIG_DIR/'top10_productos_notebook.png')

# Distribuciones por producto (histograma de medias)
product_stats = df_merged.groupby('id_producto').agg({
    target_cols['precio_unitario']: 'mean',
    target_cols['cantidad']: 'sum',
    target_cols['total_venta']: 'sum'
}).reset_index()

plt.figure(figsize=(10,6))
sns.histplot(product_stats[target_cols['total_venta']], kde=True)
plt.title('Distribución de Venta Total por Producto (suma)')
plt.tight_layout()
plt.savefig(FIG_DIR/'distribucion_venta_por_producto_notebook.png', dpi=200)
plt.show()
print('Guardado:', FIG_DIR/'distribucion_venta_por_producto_notebook.png')

In [ ]:
# 📝 10) Generar resumen ejecutiva automatizado (Markdown)
report_lines = []
report_lines.append('# Resumen Ejecutivo - Análisis de Ventas')
report_lines.append('\n## Métricas clave')
if 'venta_total' in target_cols.values() or 'total_venta' in target_cols.values():
    total_ventas = df_merged[target_cols['total_venta']].sum()
    report_lines.append(f'- Ventas totales analizadas: {int(total_ventas)}')
report_lines.append('\n## Conclusiones rápidas')
report_lines.append('- El precio unitario es un driver importante de ingresos.')
report_lines.append('- Existe concentración de ingresos en pocos productos.')
report_lines.append('- Se recomienda RFM y forecasting para abastecimiento.')

report_md = REPORTS_DIR / 'reporte_ejecutivo_notebook.md'
report_md.write_text('\n'.join(report_lines), encoding='utf-8')
print('Reporte guardado en:', report_md)


---

## ✅ Conclusiones y Siguiente Pasos (para presentar)

- **Conclusiones cualitativas**: el negocio combina volumen con productos premium; hay patrones de compra repetitivos y compras masivas puntuales.
- **Implicaciones estratégicas**: optimizar precios de productos clave, proteger inventario, diseñar promociones por ciclo (fin de mes, fines de semana).
- **Métricas calculadas**: correlaciones, estadísticos descriptivos, concentración Top10, outliers (IQR), ventas mensuales.
- **Próximos pasos sugeridos**: RFM, forecasting avanzado (Prophet/ARIMA), dashboard interactivo.

---

Notebook creado para presentación. Guarda este archivo en la raíz del proyecto y ejecútalo desde allí para que las rutas relativas funcionen correctamente.